In [1]:
from os.path import dirname, join, isfile, isdir
from datetime import datetime, timedelta
from unidecode import unidecode
from hashlib import sha256
from os import makedirs
from sys import exit
import pandas as pd
import codecs


from bulletin import __file__ as __root__
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.data.notifica import Notifica
from bulletin.commom import static
from bulletin.commom.normalize import normalize_text, normalize_labels, normalize_number, normalize_municipios, normalize_igbe, normalize_hash, data_hash
from bulletin.commom.utils import Timer, auto_fit_columns

pd.set_option('display.max_columns', None)
output = join("output","correcoes","casos_ativos")


if not isdir(output):
    makedirs(output)

criou


In [26]:
ativos = pd.read_excel(join("input","ativos.xlsx"), 
    converters = {
        'IBGE_RES_PR': normalize_igbe,
        'Nome': normalize_text,
        'Sexo': normalize_text,
        'Idade': lambda x: normalize_number(x,fill=0),
        'Mun Resid': normalize_text,
        'Mun atend': normalize_text,
        'RS': lambda x: normalize_number(x,fill=99),
        'Laboratório': normalize_text },
    parse_dates=False )

ativos.columns = [ normalize_labels(x) for x in ativos.columns ]
ativos = ativos.rename(columns={'ibge_res_pr': 'ibge'})

ativos['rs'] = ativos['rs'].apply(lambda x: str(x).zfill(2) if x != 99 else None)

ativos['ordem'] = [ x for x in range(1,len(ativos)+1) ]
ativos = ativos[['ordem','rs','ibge','mun_resid','nome','sexo','idade','mun_atend','dt_notificacao','dias_apos_notificacao']]

In [30]:
for rs, df_rs in ativos.groupby('rs'):
    writer = pd.ExcelWriter(join(output,f"ativos_{int(rs)}rs.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

    df_rs = df_rs.sort_values('dias_apos_notificacao',ascending=False)
    df_rs.to_excel(writer,'geral',index=False)
    worksheet = writer.sheets[f"geral"]
    auto_fit_columns(worksheet,df_rs)

    for ibge, df_ibge in df_rs.groupby('ibge'):
        df_ibge.to_excel(writer,f"{ibge}",index=False)
        worksheet = writer.sheets[f"{ibge}"]
        auto_fit_columns(worksheet,df_ibge)

    writer.save()
    writer.close()

In [27]:
ativos.groupby('rs')[['ordem']].count()

,ordem
rs,
01,5472
02,30905
03,11590
04,1512
05,4861
06,1041
07,2122
08,2509
09,7198
